In [1]:
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def geoplot_tmr(br_mun, df_tmr_mun2, year):
    br_mun_tmr_mun_year = br_mun.merge(
        df_tmr_mun2.query(f"ano_referencia == {year}"),
        how="left",
        left_on="CD_MUN",
        right_on="id_municipio",
    )
    f, ax = plt.subplots(figsize=(16, 16))
    br_mun.plot(ax=ax, color="#ddd")  # Base
    br_mun_tmr_mun_year.plot(
        ax=ax,
        column="tarifa_media_trend",
        cmap="Blues",
    )
    ax.set_xlim(-75, -33)
    ax.set_ylim(-35, 6)
    ax.text(
        x=-70,
        y=-25,
        s=f"{year}",
        fontsize=90,
    )
    f.tight_layout()
    ax.axis("off")
    return f, ax

In [4]:
plots_dir = Path("img", "maps")
plots_dir.mkdir(parents=True, exist_ok=True)
data_dir = Path("data")

br_mun = gpd.read_file(data_dir / "BR_Municipios_2021.zip")
br_mun = br_mun.assign(CD_MUN=lambda x: x["CD_MUN"].astype("int64"))
df_tmr_mun3 = pd.read_csv(data_dir / "processed_tmr_mun.csv")

In [5]:
# CONFIGURATION
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 16

In [6]:
for i, year in enumerate(range(1995, 2021 + 1)):
    print(year, end=" ")
    f, ax = geoplot_tmr(br_mun, df_tmr_mun3, year=year)
    f.savefig(plots_dir / f"{i:03}.png", dpi=300)
    plt.close()

1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 

Create GIF

In [ ]:
# https://stackoverflow.com/questions/24961127/how-to-create-a-video-from-images-with-ffmpeg
!ffmpeg -framerate 1 -pattern_type glob -i 'img/maps/*.png' \
  -vf "fps=10,scale=500:-1:flags=lanczos,split[s0][s1];[s0]palettegen[p];[s1][p]paletteuse" \
  -loop 0 animation.gif